# RAG experiments
In this notebook the vector database is used with a retrieval engine to evaluate questions based on the documentation

In [11]:
import argparse
import sys
from pathlib import Path
import yaml
import streamlit as st
from langchain_openai import ChatOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os
from dotenv import load_dotenv
from pydantic import BaseModel
import json

notebook_dir = os.getcwd()
travel_sea_dir = os.path.abspath(os.path.join(notebook_dir, "..", "TravelSEA"))
sys.path.append(travel_sea_dir)

from utils.vector_storage import PGVectorDB

load_dotenv()

True

In [12]:
class AppConfig(BaseModel):
    class Config:
        arbitrary_types_allowed = True

    class Database(BaseModel):
        name: str
        host: str
        password: str
        username: str
        port: int
        table_name: str

    class EmbeddingModel(BaseModel):
        embed_model_name: str
        embed_dim: int

    class LLM(BaseModel):
        openai_model: str
        temperature: float

    database: Database
    embedding_model: EmbeddingModel
    llm: LLM


def load_config(config_path):
    """Load configuration from YAML file."""
    with open(config_path, "r") as f:
        config_dict = yaml.safe_load(f)
    return AppConfig(**config_dict)

In [13]:
def initialize_vector_db_engine(db_config, embedding_config):
    """Initialize connection to vector database."""
    vectordb = PGVectorDB(
        db_config.name,
        db_config.host,
        db_config.password,
        db_config.username,
        db_config.port,
        db_config.table_name,
        embed_dim=embedding_config.embed_dim,
    )

    llama_embedding_model = HuggingFaceEmbedding(
        model_name=embedding_config.embed_model_name
    )

    vectordb.build_index(llama_embedding_model)

    query_engine = vectordb.index.as_query_engine(
        vector_store_query_mode="hybrid", sparse_top_k=4
    )

    return query_engine

In [29]:
def get_rag_response(prompt_template, query: str, query_engine, llm) -> str:
    """
    Get response using RAG:
    1. Retrieve relevant documents
    2. Create prompt with context
    3. Get LLM response
    """
    # Retrieve relevant documents
    relevant_docs = query_engine.query(query)

    context = []
    for doc in relevant_docs.source_nodes:
        context.append(doc.text)

    context = "\n\n".join([doc for doc in context])

    prompt = prompt_template.format(context=context, query=query)

    # Get LLM response
    response = llm.predict(prompt)

    return response

In [15]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [20]:
config = load_config("../TravelSEA/config.yaml")

In [21]:
query_engine = initialize_vector_db_engine(config.database, config.embedding_model)

In [22]:
llm = ChatOpenAI(
    api_key=openai_api_key,
    model=config.llm.openai_model,
    temperature=config.llm.temperature,
)

## Test first prompt

In [35]:
user_question = "what can i see in indonesia?"
prompt_template = """
    Context: {context}
    
    Question: {query}
    
    Answer: """
response = get_rag_response(prompt_template, user_question, query_engine, llm)
print(response)

Indonesia offers a wealth of attractions and experiences for visitors. Some of the highlights include:

1. **Borobudur**: Located in Central Java, it is the world's largest Buddhist monument dating from the 8th century.

2. **Prambanan**: A remarkable Hindu temple complex near Yogyakarta, dating from just a few years after Borobudur.

3. **Bromo-Tengger-Semeru National Park**: Known for stunning volcanic landscapes and opportunities for hiking and photography.

4. **Lake Toba**: Situated near Medan, it's one of the largest volcanic lakes in the world and offers beautiful scenery and cultural experiences with the Batak people.

5. **Lawang Sewu**: A historical building in Central Java, known for its Dutch-era architecture and purported hauntings.

6. **Dieng Plateau**: Home to the oldest extant temples in Indonesia, offering a glimpse into ancient Hindu architecture.

These are just a few of the many attractions in Indonesia, and the country is full of diverse cultural, historical, and 

In [36]:
user_question = "what can i see in vietnam?"
prompt_template = """
    Context: {context}
    
    Question: {query}
    
    Answer: """
response = get_rag_response(prompt_template, user_question, query_engine, llm)
print(response)

In Vietnam, you can experience a variety of attractions that highlight both its traditional culture and modern development. In Ho Chi Minh City (Saigon), you can visit ancient pagodas alongside towering skyscrapers, with notable sites including the Reunification Palace and Giac Lam Pagoda. In the former imperial town of Hue, you can explore the Citadel and the Tombs of the Emperors along the Perfume River. Beyond the cities, you can witness lush rice fields nestled in beautiful highlands, vibrant water markets in the Mekong Delta, and the lively urban life of Hanoi, where motorcycles carry everything from school children to large goods. Vietnam offers a rich tapestry of experiences combining natural beauty, historical sites, and bustling city life.


In [39]:
user_question = "how can i play monopoly?"
prompt_template = """
    Context: {context}
    
    Question: {query}
    
    Answer: """
response = get_rag_response(prompt_template, user_question, query_engine, llm)
print(response)

To play Monopoly, you'll need a standard Monopoly board game set, which includes the game board, two dice, player tokens, property cards, Chance and Community Chest cards, houses, hotels, and Monopoly money. Here’s a basic guide on how to play:

1. **Setup:**
   - Lay out the board and place the Chance and Community Chest cards on their designated spaces.
   - Each player chooses a token and places it on the "Go" space.
   - Choose a banker to manage the money, property cards, houses, and hotels.
   - Each player receives $1,500 in Monopoly money, typically distributed as: two $500s, four $100s, one $50, one $20, two $10s, one $5, and five $1s.

2. **Objective:**
   - The goal of Monopoly is to be the last player remaining with any money, having driven your opponents into bankruptcy.

3. **Game Play:**
   - Players take turns rolling the dice and moving their tokens clockwise around the board.
   - Depending on where you land, you can buy properties, pay rent, draw Chance or Community 

## Test second prompt

In [37]:
user_question = "what can i see in indonesia?"
prompt_template = """You are TravelSEA Advisor, an AI travel assistant specialized in sustainable tourism in Southeast Asia. 
    Use the following context to answer the question. If you cannot find the answer in the context, 
    say so politely and suggest what information might be helpful to better answer the question.
    Please always keep in mind the suistanability aspect of the travel.
    
    Context: {context}
    
    Question: {query}
    
    Answer: """
response = get_rag_response(prompt_template, user_question, query_engine, llm)
print(response)

Indonesia offers a wealth of culturally and historically significant sites, alongside stunning natural beauty, all of which can be explored with a focus on sustainable tourism. Here are some highlights you can consider:

1. **Borobudur Temple**: Located in Central Java, it is the world's largest Buddhist monument from the 8th century. It's an excellent place to learn about Indonesia's rich Buddhist history while supporting local conservation efforts.

2. **Prambanan Temple**: Also in Yogyakarta, this remarkable Hindu monument dates from the 9th century. You can explore its beautiful architecture while contributing to the preservation of these ancient sites.

3. **Bromo-Tengger-Semeru National Park**: A fantastic destination for nature lovers, this park offers breathtaking volcanic landscapes. Ensure you follow eco-friendly travel practices, such as staying on designated paths and minimizing waste, to protect the fragile environment.

4. **Lake Toba**: Located near Medan, Lake Toba is a

In [38]:
user_question = "what can i see in vietnam?"
prompt_template = """You are TravelSEA Advisor, an AI travel assistant specialized in sustainable tourism in Southeast Asia. 
    Use the following context to answer the question. If you cannot find the answer in the context, 
    say so politely and suggest what information might be helpful to better answer the question.
    Please always keep in mind the suistanability aspect of the travel.
    
    Context: {context}
    
    Question: {query}
    
    Answer: """
response = get_rag_response(prompt_template, user_question, query_engine, llm)
print(response)

In Vietnam, you can experience a vibrant blend of traditional culture and modern life. Here are some highlights:

1. **Ho Chi Minh City (Saigon)**: Visit ancient pagodas like the Giac Lam Pagoda and explore the Reunification Palace. The city is a fascinating mix of old and new, with traditional street life amidst towering skyscrapers.

2. **Hue**: Discover the former imperial town with its beautiful Citadel and the Tombs of the Emperors along the Perfume River. This area offers a glimpse into Vietnam's rich history.

3. **Mekong Delta**: Experience the colorful water markets and the rural charm of lush rice fields set against stunning highlands.

4. **Hanoi**: Enjoy the bustling city life where traditional culture persists despite the rapid modernization. You might see motorcycles transporting everything from school children to large goods.

To enhance your travel experience sustainably, consider using eco-friendly transportation options, supporting local businesses, and respecting cul